In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import joblib
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
train_data = pd.read_excel('train_data.xlsx')

In [4]:
# 1. Address missing values
train_data.fillna(train_data.median(numeric_only=True), inplace=True)
train_data.fillna("Unknown", inplace=True)

In [5]:
# 2. Encode categorical variables
categorical_columns = ['purpose', 'home_ownership', 'verification_status', 'application_type']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le


In [6]:

# 3. Normalize numeric variables
scaler = StandardScaler()
numeric_columns = ['annual_inc', 'loan_amnt', 'account_bal', 'installment', 'cibil_score']
train_data[numeric_columns] = scaler.fit_transform(train_data[numeric_columns])


In [7]:
# 4. Create derived features
train_data['debt_to_income_ratio'] = train_data['installment'] / (train_data['annual_inc'] + 1e-6)


In [8]:
X = train_data.drop(['loan_status', 'customer_id', 'transaction_date','term','sub_grade','home_ownership','total_no_of_acc','purpose','application_type','verification_status','installment',"account_bal"], axis=1)
y = train_data['loan_status']

In [9]:
# Handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [11]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [12]:
# 6. Train a Random Forest model
rf_model = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10, class_weight='balanced')
rf_model.fit(X_train, y_train)


RandomForestClassifier(class_weight='balanced', max_depth=10, random_state=42)

In [13]:
# 7. Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [14]:
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

Accuracy: 0.73

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.71      0.72     16996
           1       0.71      0.75      0.73     16611

    accuracy                           0.73     33607
   macro avg       0.73      0.73      0.73     33607
weighted avg       0.73      0.73      0.73     33607


Confusion Matrix:
 [[11994  5002]
 [ 4158 12453]]


In [15]:
from sklearn.linear_model import LogisticRegression
rf_model = LogisticRegression()
rf_model.fit(X_train, y_train)



LogisticRegression()

In [16]:
# 7. Evaluate the model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

In [17]:
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)

joblib.dump(rf_model, 'logistic_regression_model.pkl')

Accuracy: 0.65

Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.60      0.63     16996
           1       0.63      0.70      0.66     16611

    accuracy                           0.65     33607
   macro avg       0.65      0.65      0.65     33607
weighted avg       0.65      0.65      0.65     33607


Confusion Matrix:
 [[10162  6834]
 [ 5003 11608]]


['logistic_regression_model.pkl']

In [20]:
test_data = pd.read_excel('test_data.xlsx')

# Preprocess the test data (similar to the train data preprocessing)
test_data['debt_to_income_ratio'] = test_data['installment'] / (test_data['annual_inc'] + 1e-6)
X_new = test_data.drop(['loan_status', 'customer_id', 'transaction_date','term','sub_grade','home_ownership','total_no_of_acc','purpose','application_type','verification_status','installment',"account_bal"], axis=1)

# Load the saved model
model = joblib.load('logistic_regression_model.pkl')

# Make predictions on the new data
predictions = model.predict(X_new)

# Optionally, if the 'loan_status' column exists in the test data, calculate accuracy
if 'loan_status' in test_data.columns:
    accuracy = accuracy_score(test_data['loan_status'], predictions)
    print(f"Accuracy: {accuracy:.2f}")
else:
    print("Loan status is not available in the test data. Predictions made but accuracy cannot be calculated.")

Accuracy: 0.64


**bold text**

### Chosen Model: RANDOM FOREST
Accuracy: 0.73

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.71      0.72     16996
           1       0.71      0.75      0.73     16611

    accuracy                           0.73     33607
   macro avg       0.73      0.73      0.73     33607
weighted avg       0.73      0.73      0.73     33607

This indicates that the model correctly predicted the class labels for 73% of the instances in the dataset.

The model demonstrates a relatively balanced performance across both classes, with accuracy, precision, recall, and F1-scores all around 0.73.

Class 1 exhibits slightly higher recall compared to Class 0, suggesting that the model might be better at identifying true Class 1 instances.
Class 0 has slightly higher precision, indicating that when the model predicts Class 0, it tends to be more accurate.